In [2]:
from pdf2image import convert_from_path

pdf_path = 'metro.pdf'
dpi = 600

images = convert_from_path(pdf_path, dpi=dpi)

images[0].save(f'metro.png', 'PNG')

In [13]:
from PIL import Image
import os
import numpy as np

binary_mask = Image.open('shape.png')
binary_mask = binary_mask.convert('1') 

image_folder = 'icons/'

output_folder = 'processed/'
os.makedirs(output_folder, exist_ok=True)

png_images = [f for f in os.listdir(image_folder) if f.endswith('.png')]

binary_mask = np.array(binary_mask)

for image_filename in png_images:
    image_path = os.path.join(image_folder, image_filename)
    image = Image.open(image_path)

    image_array = np.array(image)

    result_image = np.dstack([image_array[:, :, 0] * binary_mask, image_array[:, :, 1] * binary_mask, image_array[:, :, 2] * binary_mask])

    result_image = Image.fromarray(result_image)

    gray_image = result_image.convert('L')
    threshold = 200

    binary_image = gray_image.point(lambda p: p > threshold and 255)

    result_filename = os.path.join(output_folder, image_filename)
    binary_image.save(result_filename)

print("[ All images have been processed ]")

[ All images have been processed ]


In [15]:
import cv2
import os
import numpy as np

# Define the folder containing your images
input_folder = 'processed'
output_folder = 'augmented_images'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# List of augmentation functions (you can add more or modify them as needed)
def augment_image(image):
    # Apply a random combination of augmentations
    random_seed = np.random.randint(1, 100)
    random = np.random.RandomState(random_seed)

    # Rotate by a random angle
    angle = random.uniform(-10, 10)
    image = cv2.warpAffine(image, cv2.getRotationMatrix2D((133 / 2, 133 / 2), angle, 1), (133, 133))

    # Randomly flip horizontally
    if random.rand() < 0.5:
        image = np.fliplr(image)

    # Randomly adjust brightness and contrast
    alpha = 1 + random.uniform(-0.2, 0.2)
    beta = 0 + random.uniform(-20, 20)
    image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

    # Randomly add Gaussian noise
    noise = np.random.normal(0, 20, image.shape).astype(np.uint8)
    image = cv2.add(image, noise)

    return image

# Loop through the images in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Load the image
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)

        # Perform data augmentation
        augmented_image = augment_image(image)

        # Save the augmented image to the output folder
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, augmented_image)

print("[ Data augmentation complete ]")


[ Data augmentation complete ]
